In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
from agents import Agent, Runner,function_tool,trace, ItemHelpers, MessageOutputItem
import nest_asyncio
nest_asyncio.apply()
# setting up api key
os.environ["OPENAI_API_KEY"] = os.environ['OPENAI_API_KEY_MALI']
import wikipedia
import openmeteo_requests
import requests_cache
# from retry_requests import retry
import requests
from agents.extensions.visualization import draw_graph

import json 
import requests
from serpapi import GoogleSearch
api_key = os.getenv("SERPAPI_API_KEY")


from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,  # tool call streaming
    ResponseCreatedEvent,  # start of new event like tool call or final answer
)

from tools import flight_search, yelp_search, hotels_search, wikipedia_search, get_weather
from pydantic import BaseModel
from models import first_agent_output
from agents.model_settings import ModelSettings
from data_models.flight_data_models import FlightSearchResults, BestFlight, AirportInfo, Flight, Layover, OtherFlight, PriceInsights, AirportLocation, Airports
from data_models.yelp_data_models import Category, Business, YelpSearchResults
from travel_agents import question_rewrite_agent, flight_agent
from functions import flight_search,yelp_search, hotels_search
from pprint import pprint
from data_models.hotel_data_models import HotelSearchResponse, SearchInformation, Brand, BrandChild, Property, GPSCoordinates, ReviewsBreakdown, Rating, Price, Rate, NearbyPlace, Transportation

END OF THE SCRIPT


In [2]:
ank = "ankara(ESB)"
ist = "istanbul(SAW)"
amst = "amsterdam(AMS)"
pekin = "Peking(PEK)"

start_point = ank 
dest_point = amst

main_question = f"""I want to travel from {start_point} to {dest_point}. The outbound date is 2025-05-05 and the return date will be 2025-05-11. 
I also need a hotel to stay. I do not like smell of cigarette. If possible, I want to see the rooms where smoking is not allowed. 
In addition to that, I like eating sea food. I guess {dest_point} is very rich city interms of sea restaurants""".strip()

question_rewrite_agent_output = Runner.run_sync(question_rewrite_agent,main_question)

In [3]:
question_rewrite_agent_output.final_output.hotel_reqs

HotelOutput(query='Amsterdam', check_in_date='2025-05-05', check_out_date='2025-05-11', gl='non-smoking')

In [4]:
x = hotels_search(q= question_rewrite_agent_output.final_output.hotel_reqs.query, 
              check_in_date= question_rewrite_agent_output.final_output.hotel_reqs.check_in_date,
              check_out_date =question_rewrite_agent_output.final_output.hotel_reqs.check_out_date, 
              gl = 'nl'
              )

In [30]:
hotels_info = ""
for i in range(len(x['properties'])):
    test_output = Property(**x['properties'][i])
    t_data = f"""

{i+1}
Name : {test_output.name}
check-in : {test_output.check_in_time}
check-out : {test_output.check_out_time}
lowest price per night : {test_output.rate_per_night.lowest}
overall_rating : {test_output.overall_rating}
amenities : {test_output.amenities}"""
    hotels_info += t_data

In [31]:
print(hotels_info)



1
Name : 25m2 Studio | Near Amsterdam & Schiphol
check-in : 3:00 PM
check-out : 11:00 AM
lowest price per night : US$134
overall_rating : 4.2
amenities : ['Air conditioning', 'Kid-friendly', 'Elevator', 'Fitness center', 'Heating', 'Kitchen', 'Microwave', 'Smoke-free', 'Cable TV', 'Washer', 'Free parking', 'Free Wi-Fi']

2
Name : via Amsterdam
check-in : 3:00 PM
check-out : 10:00 AM
lowest price per night : US$107
overall_rating : 4.3
amenities : ['Breakfast', 'Free Wi-Fi', 'Parking', 'Air conditioning', 'Bar', 'Restaurant', 'Kitchen in some rooms', 'Child-friendly', 'Smoke-free property']

3
Name : Zoku Amsterdam
check-in : 3:00 PM
check-out : 11:00 AM
lowest price per night : US$416
overall_rating : 4.5
amenities : ['Air conditioning', 'Beach access', 'Kid-friendly', 'Crib', 'Elevator', 'Heating', 'Indoor pool', 'Ironing board', 'Kitchen', 'Microwave', 'Outdoor pool', 'Oven stove', 'Pet-friendly', 'Smoke-free', 'Cable TV', 'Washer', 'Wheelchair accessible', 'Free parking', 'Free Wi

In [32]:
yelp_agent = Agent(
    name = "Hotels Info",
    instructions=f"You are helpful asisstants try to answer the questions according to: \n\n{hotels_info}. You are not allowed to recommend or answer any other thing than given information",
    model = 'gpt-4o-mini',
    # tools=[yelp_search],
    model_settings=ModelSettings(temperature= 0.0,
                                 max_tokens = 4096*2),
)

result = Runner.run_sync(yelp_agent, main_question+" recommend the cheapest accomodations")
print(result.final_output)

Here are some hotel options in Amsterdam that are smoke-free and have the lowest prices:

1. **MEININGER Hotel Amsterdam City West**
   - **Check-in:** 3:00 PM
   - **Check-out:** 11:00 AM
   - **Lowest price per night:** US$125
   - **Overall rating:** 4.2
   - **Amenities:** Breakfast ($), Free Wi-Fi, Parking ($), Pet-friendly, Bar, Accessible, Child-friendly, Smoke-free property.

2. **Camping Zeeburg Amsterdam**
   - **Check-in:** 2:00 PM
   - **Check-out:** 10:00 AM
   - **Lowest price per night:** US$37
   - **Overall rating:** 4.5
   - **Amenities:** Free breakfast, Free Wi-Fi, Free parking, Pet-friendly, Bar, Restaurant, Kitchen in some rooms, Airport shuttle, Child-friendly.

3. **Amrâth Apart-Hotel Schiphol Badhoevedorp**
   - **Check-in:** 3:00 PM
   - **Check-out:** 11:00 AM
   - **Lowest price per night:** US$131
   - **Overall rating:** 4.4
   - **Amenities:** Free Wi-Fi, Parking ($), Air conditioning, Fitness centre, Restaurant, Kitchen in rooms, Airport shuttle, Full-se

In [33]:
yelp_agent = Agent(
    name = "Hotels Info",
    instructions=f"You are helpful asisstants try to answer the questions according to: \n\n{hotels_info}. You are not allowed to recommend or answer any other thing than given information",
    model = 'gpt-4o-mini',
    # tools=[yelp_search],
    model_settings=ModelSettings(temperature= 0.0,
                                 max_tokens = 4096*2),
)

result = Runner.run_sync(yelp_agent, main_question+" recommend the cheapest accomodations and sort it from cheapest the to exps")
print(result.final_output)

Here are the hotel options in Amsterdam that are smoke-free and sorted from cheapest to most expensive:

1. **Camping Zeeburg Amsterdam**
   - **Check-in:** 2:00 PM
   - **Check-out:** 10:00 AM
   - **Lowest price per night:** US$37
   - **Overall rating:** 4.5
   - **Amenities:** Free breakfast, Free Wi-Fi, Free parking, Pet-friendly, Bar, Restaurant, Kitchen in some rooms, Airport shuttle, Child-friendly

2. **MEININGER Hotel Amsterdam City West**
   - **Check-in:** 3:00 PM
   - **Check-out:** 11:00 AM
   - **Lowest price per night:** US$125
   - **Overall rating:** 4.2
   - **Amenities:** Breakfast ($), Free Wi-Fi, Parking ($), Pet-friendly, Bar, Accessible, Child-friendly, Smoke-free property

3. **Amrâth Apart-Hotel Schiphol Badhoevedorp**
   - **Check-in:** 3:00 PM
   - **Check-out:** 11:00 AM
   - **Lowest price per night:** US$131
   - **Overall rating:** 4.4
   - **Amenities:** Free Wi-Fi, Parking ($), Air conditioning, Fitness centre, Restaurant, Kitchen in rooms, Airport shu